In [5]:
import rdflib
from rdflib import Graph, URIRef, Literal, BNode
from pandas import DataFrame
from rdflib.plugins.sparql.processor import SPARQLResult, prepareQuery, prepareUpdate
from rdflib.namespace import FOAF, RDF, SDO
from rdflib.namespace import DefinedNamespace, Namespace
from rdflib.term import URIRef

import numpy as np
from random import sample, choice

In [6]:
def sparql_results_to_df(results: SPARQLResult) -> DataFrame:
    """
    Export results from an rdflib SPARQL query into a `pandas.DataFrame`,
    using Python types. See https://github.com/RDFLib/rdflib/issues/1179.
    """
    return DataFrame(
        data=([None if x is None else x.toPython() for x in row] for row in results),
        columns=[str(x) for x in results.vars],
    )

In [7]:
class ErrorRDF(RDF):
    _NS = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-nsERROR#")

class ErrorFOAF(FOAF):
    _NS = Namespace("http://xmlns.com/foaf/0.1/ERROR/")

class ErrorSDO(SDO):
    _NS = Namespace("https://schema.org/ERROR/")

    Human: URIRef 
    HumAn: URIRef 
    homeAddress: URIRef
    

In [8]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("error_foaf", ErrorFOAF)
g.bind("sdo", SDO)
g.bind("error_sdo", ErrorSDO)

bob = URIRef("http://example.org/people/Bob")
linda = URIRef("http://example.org/people/Linda")
error_hans = URIRef("http://exampleorg/Hans")

g.add((bob, RDF.type, FOAF.Person))
g.add((bob, FOAF.name, Literal("Bob")))
g.add((bob, ErrorFOAF.age, Literal(24)))
g.add((bob, FOAF.knows, linda))
g.add((linda, RDF.type, FOAF.Person))
g.add((linda, FOAF.name, Literal("Linda")))
g.add((error_hans, RDF.type, ErrorSDO.HumAn))
g.add((error_hans, ErrorSDO.name, Literal("Hans")))

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://exampleorg/Hans> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> a foaf:Person ;
    foaf:name "Linda" .




In [9]:
q = prepareUpdate(
    """DELETE {
        ?source_id ?p ?o .
    }
    INSERT { 
        ?target_id ?p ?o . 
    }
    WHERE {
        ?source_id ?p ?o .
    }"""
)
g.update(q, initBindings={'source_id': rdflib.URIRef("http://exampleorg/Hans"), 'target_id': rdflib.URIRef("http://exampleorg/Hans1")})
print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://exampleorg/Hans1> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> a foaf:Person ;
    foaf:name "Linda" .




In [10]:
qres = g.query(
        """
        SELECT DISTINCT ?s
        WHERE {
            ?s ?p ?o .
        }
        """
    )

for row in qres:
    print(f"{row.s}")

sparql_results_to_df(qres)

http://example.org/people/Linda
http://example.org/people/Bob
http://exampleorg/Hans1


,s
0,http://example.org/people/Linda
1,http://example.org/people/Bob
2,http://exampleorg/Hans1


In [11]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        PREFIX ex: <http://example.org/people/>
        DELETE {
            ex:Linda a ?o .
        }
        INSERT { 
            ex:Linda1 a ?o . 
        }
        WHERE {
            ex:Linda a ?o .
        }
        """
    )

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a foaf:Person .

<http://exampleorg/Hans1> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [12]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        DELETE { 
            ?s foaf:knows ?o . 
        }
        INSERT { 
            ?s error_foaf:knows ?o . 
        }
        WHERE {
            ?s foaf:knows ?o .
        }
        """
    )
print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    error_foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a foaf:Person .

<http://exampleorg/Hans1> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [13]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        DELETE {
            ?s a foaf:Person .
        }
        INSERT { 
            ?s a error_foaf:Human . 
        }
        WHERE {
            ?s a foaf:Person .
        }
        """
    )

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a error_foaf:Human ;
    error_foaf:age 24 ;
    error_foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a error_foaf:Human .

<http://exampleorg/Hans1> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [14]:
semicolon_pos = [pos for pos, char in enumerate(str(g.serialize())) if char == ';']
dot_pos = [pos for pos, char in enumerate(str(g.serialize())) if char == '.']
str_arr = np.array(list(str(g.serialize())))
str_arr[sample(semicolon_pos, 2)] = 'Ä'
str_arr[sample(dot_pos, 2)] = 'Ö'
str("".join(str_arr))

'@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .\n@prefix error_sdo: <https://schema.org/ERROR/> .\n@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<http://exampleÖorg/people/Bob> a error_foaf:Human Ä\n    error_foaf:age 24 ;\n    error_foaf:knows <http://example.org/people/Linda> Ä\n    foaf:name "Bob" .\n\n<http://example.org/people/Linda1> a error_foaf:Human .\n\n<http://exampleorg/Hans1> a error_sdo:HumAn ;\n    error_sdo:name "Hans" .\n\n<http://example.org/people/Linda> foaf:name "Linda" Ö\n\n'

In [15]:
'''q = prepareUpdate(
    "INSERT DATA { <?person1> foaf:knows <?person2> .}",
    initNs = { "foaf": FOAF }
)

tim = rdflib.URIRef("http://www.w3.org/People/Berners-Lee/card#i")
g.update(q, initBindings={'person1': tim, 'person2': bob})

print(g.serialize())'''

'q = prepareUpdate(\n    "INSERT DATA { <?person1> foaf:knows <?person2> .}",\n    initNs = { "foaf": FOAF }\n)\n\ntim = rdflib.URIRef("http://www.w3.org/People/Berners-Lee/card#i")\ng.update(q, initBindings={\'person1\': tim, \'person2\': bob})\n\nprint(g.serialize())'

In [16]:
# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
#g.parse("https://makg.org/samples/2018-11-09/Affiliations_s.nt")
g.parse("http://www.w3.org/People/Berners-Lee/card")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("No triple found!")
    else:
       print(subj, pred, obj)

https://www.w3.org/People/Berners-Lee/card#i http://xmlns.com/foaf/0.1/account http://twitter.com/timberners_lee
https://www.w3.org/People/Berners-Lee/card#i http://www.w3.org/ns/solid/terms#editableProfile https://timbl.com/timbl/Public/friends.ttl
n3d5c2560074f40a4b4dc587688679089b2 http://www.w3.org/2000/10/swap/pim/contact#address n3d5c2560074f40a4b4dc587688679089b3
https://www.w3.org/People/Berners-Lee/card#i http://www.w3.org/ns/pim/space#storage https://timbl.solid.community/
https://www.w3.org/People/Berners-Lee/card#i http://www.w3.org/2000/10/swap/pim/contact#office n3d5c2560074f40a4b4dc587688679089b2
http://dig.csail.mit.edu/data#DIG http://xmlns.com/foaf/0.1/member https://www.w3.org/People/Berners-Lee/card#i
https://timbl.com/timbl/Public/friends.ttl http://xmlns.com/foaf/0.1/primaryTopic https://www.w3.org/People/Berners-Lee/card#i
http://www.w3.org/People/Berners-Lee/card http://xmlns.com/foaf/0.1/maker https://www.w3.org/People/Berners-Lee/card#i
https://www.w3.org/Peop

In [17]:
g.serialize(destination="test.ttl", format="turtle")

g = Graph()
g.parse("test.ttl")
print(f"Graph g has {len(g)} statements.")

Graph g has 86 statements.


In [18]:
from rdflib.namespace import FOAF, RDF, SDO, RDFS, OWL
from rdflib import Graph, URIRef, Literal, BNode
from framework.namespaces import ErrorFOAF, ErrorRDF, ErrorSDO, Example

g = Graph()
g.bind("foaf", FOAF)
g.bind("sdo", SDO)
g.bind("rdfs", RDFS)
g.bind("owl", OWL)
g.bind("ex", Example)

dennis = URIRef("http://example.org/Dennis")
dennis_som = URIRef("http://example.org/DennisSom")

g.add((dennis, RDF.type, FOAF.Person))
g.add((dennis, RDF.type, SDO.Person))
g.add((dennis, FOAF.name, Literal("Dennis")))
g.add((dennis, FOAF.age, Literal(24)))

g.add((Example.DAddress, RDF.type, SDO.PostalAddress))
g.add((Example.DAddress, SDO.postalCode, Literal("6020")))
g.add((dennis, SDO.address, Example.DAddress))

g.add((SDO.address, RDFS.range, SDO.PostalAddress))
g.add((Example.addressDoor, RDFS.domain, SDO.PostalAddress))

<Graph identifier=Nbfdf784430a64c5ea1e6359a66c5a0c8 (<class 'rdflib.graph.Graph'>)>

In [30]:
qres = g.query(
        """
        SELECT ?s ?o
        WHERE {
            ?s rdf:type ?o .
        }
        """,
        initNs = { "rdf": RDF }
    )

for row in qres:
    print(f"{row.s}")

sparql_results_to_df(qres)

http://example.org/Dennis
http://example.org/Dennis
http://example.org/DAddress


,s,o
0,http://example.org/Dennis,http://xmlns.com/foaf/0.1/Person
1,http://example.org/Dennis,https://schema.org/Person
2,http://example.org/DAddress,https://schema.org/PostalAddress


In [48]:
dir(SDO)

[rdflib.term.URIRef('https://schema.org/AMRadioChannel'),
 rdflib.term.URIRef('https://schema.org/APIReference'),
 rdflib.term.URIRef('https://schema.org/Abdomen'),
 rdflib.term.URIRef('https://schema.org/AboutPage'),
 rdflib.term.URIRef('https://schema.org/AcceptAction'),
 rdflib.term.URIRef('https://schema.org/Accommodation'),
 rdflib.term.URIRef('https://schema.org/AccountingService'),
 rdflib.term.URIRef('https://schema.org/AchieveAction'),
 rdflib.term.URIRef('https://schema.org/Action'),
 rdflib.term.URIRef('https://schema.org/ActionAccessSpecification'),
 rdflib.term.URIRef('https://schema.org/ActionStatusType'),
 rdflib.term.URIRef('https://schema.org/ActivateAction'),
 rdflib.term.URIRef('https://schema.org/ActivationFee'),
 rdflib.term.URIRef('https://schema.org/ActiveActionStatus'),
 rdflib.term.URIRef('https://schema.org/ActiveNotRecruiting'),
 rdflib.term.URIRef('https://schema.org/AddAction'),
 rdflib.term.URIRef('https://schema.org/AdministrativeArea'),
 rdflib.term.URIR